In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torch.optim as optim
import torchvision.models as models
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from PIL import Image

In [2]:
drive.mount('/content/drive');

Mounted at /content/drive


In [47]:
device = ('cuda' if torch.cuda.is_available() else 'cpu');
device

'cuda'

In [48]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/fashion-mnist.csv");
df.shape

(10000, 785)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:].values, df.iloc[:,0].values, test_size=0.2, random_state=42)
print(len(X_train), len(X_test))

8000 2000


In [50]:
custom_transform = transforms.Compose(
    [
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

In [51]:
class CustomDataset(Dataset):
  def __init__(self, features, labels, transform):
    self.features = features;
    self.labels = labels;
    self.transform = transform;

  def __len__(self):
    return len(self.features);

  def __getitem__(self, index):
        image = self.features[index].reshape(28,28);

        image = image.astype(np.uint8);

        image = np.stack([image]*3, axis=-1)

        image = Image.fromarray(image);

        image = self.transform(image);

        return image, torch.tensor(self.labels[index], dtype=torch.long);

In [52]:
train_dataset = CustomDataset(X_train, y_train, custom_transform);
test_dataset = CustomDataset(X_test, y_test, custom_transform);

In [53]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True);
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False);

In [54]:
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [55]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [56]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [57]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [58]:
for param in vgg16.features.parameters():
  param.requires_grad = False

In [59]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [60]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=10, bias=True)
)

In [61]:
vgg16 = vgg16.to(device);

In [62]:
learning_rate = 0.0001
epochs = 10

In [63]:
criterion = nn.CrossEntropyLoss();
optimizer =  optim.Adam(vgg16.classifier.parameters(), lr=learning_rate);

In [64]:
for epoch in range(epochs):
  total_epoch_loss = 0;
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device);

    output = vgg16(batch_features);
    loss = criterion(output, batch_labels);

    optimizer.zero_grad();

    loss.backward();

    optimizer.step();
    total_epoch_loss += loss.item();

  avg_epoch_loss = total_epoch_loss/len(train_loader);
  print("Epoch: {}/{}... ".format(epoch, epochs),
        "Loss: {:.6f}...".format(avg_epoch_loss)
       )

Epoch: 0/10...  Loss: 0.692577...
Epoch: 1/10...  Loss: 0.308472...
Epoch: 2/10...  Loss: 0.205427...
Epoch: 3/10...  Loss: 0.146361...
Epoch: 4/10...  Loss: 0.102044...
Epoch: 5/10...  Loss: 0.070170...
Epoch: 6/10...  Loss: 0.048104...
Epoch: 7/10...  Loss: 0.035642...
Epoch: 8/10...  Loss: 0.024253...
Epoch: 9/10...  Loss: 0.020123...


In [65]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [67]:
with torch.no_grad():
  total = 0
  correct = 0
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device);
    outputs = vgg16(batch_features);
    _, predicted = torch.max(outputs, 1);
    correct += (predicted == batch_labels).sum().item()
    total += batch_labels.shape[0];
  print(f"Accuracy of the transfer learning on the {X_test.shape[0]} test images: %d %%" % (100 * correct / total))

Accuracy of the transfer learning on the 2000 test images: 90 %


In [68]:
with torch.no_grad():
  total = 0
  correct = 0
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device);
    outputs = vgg16(batch_features);
    _, predicted = torch.max(outputs, 1);
    correct += (predicted == batch_labels).sum().item()
    total += batch_labels.shape[0];
  print(f"Accuracy of the transfer learning on the {X_train.shape[0]} train images: %d %%" % (100 * correct / total))

Accuracy of the transfer learning on the 8000 train images: 99 %
